In [27]:
pip install selenium

In [ ]:
# Import library 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from zipfile import ZipFile
import requests
import time
import os
import re
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 50000
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import string
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
from google.colab import files

# Membuat instance dari Chrome WebDriver dengan konfigurasi khusus
option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
option.add_argument("--headless")
option.add_argument("--disable-gpu")
option.add_argument("--no-sandbox")
option.add_argument("--remote-debugging-port=9230")
driver = webdriver.Chrome(options=option)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Scrapping URL

In [ ]:
# Membuat instance dari Chrome WebDriver dengan konfigurasi khusus
option = webdriver.ChromeOptions()
option.add_argument("--start-maximized") 
option.add_argument("--headless")   
option.add_argument("--disable-gpu")
option.add_argument("--no-sandbox") 
option.add_argument("--remote-debugging-port=9230") 
driver = webdriver.Chrome(options=option)

# Memuat halaman
driver.get("https://www.bukitvista.com/property/")

# Tunggu halaman termuat
time.sleep(3)


links = []


while True:
    property_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/property/")]')
    for link in property_links:
        href = link.get_attribute("href")
        if href and href not in links and "page" not in href:
            links.append(href)
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@aria-label, "Next")]')
        next_button.click()
        time.sleep(3) 
    except:
        break

# Masukkan tautan ke DataFrame
url_df = pd.DataFrame(links, columns=["property_links"])

# Menghapus url https://www.bukitvista.com/property/ dari DataFrame
url_df = url_df[url_df["property_links"] != "https://www.bukitvista.com/property/"].reset_index(drop=True)
driver.quit()

# Scrapping Title, Address & Tags

In [ ]:
# scraping data properti dari URL
def scrape_property_combined(url_df, container_class='container'):
    data = []

    for url in url_df["property_links"]:
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            div_containers = soup.findAll('div', attrs={'class': container_class})
            
            title = None
            address = None
            tags = None

            for container in div_containers:
                if title is None:
                    title_element = container.find(attrs={'class': 'page-title'})
                    title = title_element.text.strip() if title_element else None
                if address is None:
                    address_element = container.find(attrs={'class': 'item-address'})
                    address = address_element.text.strip() if address_element else None             
                if tags is None:
                    tags_element = container.find(attrs={'class': 'property-labels-wrap'})
                    tags = tags_element.text.strip() if tags_element else None

            # Tambahkan data hasil scraping ke list dalam bentuk dictionary
            data.append({'title': title, 'address': address, 'tags': tags})
        
        except requests.RequestException as e:
            print(f"Error accessing URL {url}: {e}")
            data.append({'title': None, 'address': None, 'tags': None})
    return pd.DataFrame(data)

# Scrape semua data properti dari DataFrame 'url_df'
property_data = scrape_property_combined(url_df)

<ipython-input-32-2c650bce8e6d>:19: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  div_containers = soup.findAll('div', attrs={'class': container_class})


# Scrapping Property Details

In [ ]:
def property_details(url, container_class='detail-wrap'):
    data = {}

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    div_container = soup.find('div', class_=container_class)
    if div_container:
        fields = div_container.find_all('li')
        for field in fields:
            strong = field.find('strong')
            span = field.find('span')
            if strong and span:
                column_name = standardize_column_name(strong.text.strip())
                data[column_name] = span.text.strip() 
    return data


def standardize_column_name(column_name):
    column_name = column_name.lower()
    replacements = {
        'bedrooms': 'bedroom',
        'bathrooms': 'bathroom',
        'rooms': 'room'
    }
    for key, value in replacements.items():
        column_name = column_name.replace(key, value)

    # Hapus karakter ':', spasi ekstra, atau simbol lainnya
    column_name = ''.join(char for char in column_name if char.isalnum() or char == ' ')
    column_name = column_name.strip().replace(' ', '_')  # Ganti spasi dengan underscore
    return column_name
all_properties = []


for index, row in url_df.iterrows():
    url = row['property_links']
    scraped_data = property_details(url)
    all_properties.append(scraped_data)


detail_properties = pd.DataFrame(all_properties)

# Scrapping Description

In [ ]:
results = []


for url in url_df["property_links"]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    container = soup.find('div', class_='block-content-wrap')

    if container:
        data = []
        current_header = None
        for element in container.find_all(['h1', 'h2', 'h3', 'p', 'li']):
            if element.name in ['h1', 'h2', 'h3']:
                current_header = element.text.strip().lower()
            elif element.name in ['p', 'li']:
                if current_header:
                    data.append({"header": current_header, "text": element.text.strip()})
                else:
                    data.append({"header": "others", "text": element.text.strip()})

        df_dict = {}
        for item in data:
            if item["header"] not in df_dict:
                df_dict[item["header"]] = []
            df_dict[item["header"]].append(item["text"])

        df_dict = {key: "\n\n".join(values) for key, values in df_dict.items()}
        results.append(df_dict)
    else:
        print(f"Container tidak ditemukan untuk URL: {url}")
        results.append({})  # Tambahkan dictionary kosong jika container tidak ditemukan

# Membuat DataFrame dari hasil
property_description = pd.DataFrame(results)

Container tidak ditemukan untuk URL: https://www.bukitvista.com/property/uluwatu-modern-boho-villa-near-nyang-nyang-beach-2


# Address Detail

In [ ]:
def address_details(url_df, container_class='list-2-cols list-unstyled'):
    data = []

    for url in url_df["property_links"]:
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            ul_container = soup.find('ul', attrs={'class': container_class})


            address = None
            city = None
            state = None
            zip_code = None
            area = None
            country = None

            if ul_container:
                for li in ul_container.find_all('li'):
                    span_element = li.find('span')
                    if span_element:
                        if 'detail-address' in li.get('class', []):
                            address = span_element.text.strip()
                        elif 'detail-city' in li.get('class', []):
                            city = span_element.text.strip()
                        elif 'detail-state' in li.get('class', []):
                            state = span_element.text.strip()
                        elif 'detail-zip' in li.get('class', []):
                            zip_code = span_element.text.strip()
                        elif 'detail-area' in li.get('class', []):
                            area = span_element.text.strip()
                        elif 'detail-country' in li.get('class', []):
                            country = span_element.text.strip()

            # Tambahkan data hasil scraping ke list dalam bentuk dictionary
            data.append({'address_detail': address, 'city': city, 'state': state, 'zip_code': zip_code, 'area': area, 'country': country})

        except requests.RequestException as e:
            print(f"Error accessing URL {url}: {e}")
            data.append({'address_detail': None, 'city': None, 'state': None, 'zip_code': None, 'area': None, 'country': None})
    return pd.DataFrame(data)

# Scrape semua data properti dari DataFrame 'url_df'
property_address_details = address_details(url_df)

# Gabung Ke dataframe

In [ ]:
# Menggabungkan beberapa DataFrame
bukit_vista_property = pd.concat([url_df, property_data, detail_properties, property_address_details], axis=1)
bukit_vista_property

,property_links,title,address,tags,property_id,price,bedroom,room,bathroom,garage,property_type,property_status,guest_number,address_detail,city,state,zip_code,area,country
0,https://www.bukitvista.com/property/uluwatu-modern-boho-villa-near-nyang-nyang-beach-2,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None
1,https://www.bukitvista.com/property/bingin-beach-hideaway-group-villa-with-pool-amp-bbq,Bingin Beach Hideaway: Group Villa with Pool & BBQ,"Jl. Pantai Cemongkak Gg. Samuh Sari No.2, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali 80361",Featured\n Bali Vacation Rental\n \n Group Friendly Stay\n \n Long Stay Trend Property\n \n New Listing\n \n Private\n \n Private Pool\n \n Top Trending\n \n New,PR0874,Starting from USD 161 per night,3,3,3,1,"Amazing pool, Island life, Pool view, Surfing, Tropical, Villa","Bali Vacation Rental, Group Friendly Stay, Long Stay Trend Property, New Listing, Private, Private Pool, Top Trending",6,"Jl. Pantai Cemongkak Gg. Samuh Sari No.2, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Bingin,Indonesia
2,https://www.bukitvista.com/property/mediterranean-luxury-villa-ubud-rooftop,4-Bedroom Mediterranean Luxury Villa with Ubud Rooftop View,"Jl. Suweta, Ubud, Kecamatan Ubud, Kabupaten Gianyar, Bali 80571",Featured\n Bali Vacation Rental\n \n Group Friendly Stay\n \n Instagramable Stay\n \n Long Stay Trend Property\n \n Private\n \n Private Pool\n \n Top Trending\n \n New,PR0960,USD 202 / Night,4,4,4,1,"Amazing pool, Amazing View, Jungle View, Pool view, Tropical, Villa","Bali Vacation Rental, Group Friendly Stay, Instagramable Stay, Long Stay Trend Property, Private, Private Pool, Top Trending",8,"Jl. Suweta, Ubud, Kecamatan Ubud, Kabupaten Gianyar, Bali",Bali,Bali,80571,Ubud,Indonesia
3,https://www.bukitvista.com/property/nusa-dua-seafront-villa,Luxurious 3-Bedroom Nusa Dua Seafront Villa w/ Private Beach,None,Featured\n Bali Vacation Rental\n \n Top Trending,PR0242,Starting from USD 715 per 2 nights,3,NaN,4,NaN,"Beachfront, Villa","Bali Vacation Rental, Top Trending",6,"Jl. Raya Nusa Dua Selatan, Sawangan, Nusa Dua, Kabupaten Badung, Bali 80363",Bali,Bali,80363,Nusa Dua,Indonesia
4,https://www.bukitvista.com/property/grand-villa-retreat-w-pool-amp-garden-in-ungasan,Grand Villa Retreat w/ Pool & Garden in Ungasan,"Jl. Pantai Balangan I No.9x, Ungasan, Kec. Kuta Sel., Kabupaten Badung, Bali 80361",Featured\n Group Friendly Stay\n \n Long Stay Trend Property\n \n New Listing\n \n Private Pool\n \n Top Trending\n \n New,PR0857,Starting from USD 84 per night,2,NaN,2,NaN,"Amazing pool, Golfing, Pool view, Villa","Group Friendly Stay, Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,"Jl. Pantai Balangan I No.9x, Ungasan, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Balangan,Indonesia
5,https://www.bukitvista.com/property/surfers-villa-4-mins-to-bingin-dreamland-beach-by-bukit-vista,Surfer’s Villa 4 Mins to Bingin & Dreamland Beach by Bukit Vista,"Pecatu, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",Featured\n Long Stay Trend Property\n \n New Listing\n \n Private Pool\n \n Top Trending,PR0925,Starting from USD 118 per night,2,NaN,2.5,NaN,"Pool view, Villa","Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,"Jl. Bangbang Metuug, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Pecatu,Indonesia
6,https://www.bukitvista.com/property/private-pool-villa-minutes-from-bingin-surf-haven-by-bukit-vista,Private Pool Villa Minutes from Bingin Surf Haven by Bukit Vista,"Pecatu, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",Featured\n Long Stay Trend Property\n \n New Listing\n \n Private Pool\n \n Top Trending,PR0925,Starting from USD 165 per night,2,NaN,2,NaN,"Pool view, Villa","Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,"Jl. Bangbang Metuug, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Pecatu,Indonesia
7,https://www.bukitvista.com/propert

# Cleaning

In [37]:
# Mengisi nilai yang hilang (NaN) pada kolom 'property_type' dengan 'Villa'
bukit_vista_property['property_type'] = bukit_vista_property['property_type'].fillna('Villa')

# Mengisi nilai yang hilang (NaN) pada kolom 'address_detail' dengan 'Uluwatu, Bali'
bukit_vista_property['address_detail'] = bukit_vista_property['address_detail'].fillna('Uluwatu, Bali')

# Mengisi nilai yang hilang (NaN) pada kolom 'price' dengan 'Start from $65 USD per night'
bukit_vista_property['price'] = bukit_vista_property['price'].fillna('Start from $65 USD per night')

In [ ]:
# Fungsi untuk mengisi kolom 'tags' berdasarkan informasi 'address_detail'
def fill_tags(row):
    if pd.isna(row['tags']):
        address_detail = str(row['address_detail']).lower() 
        if "bali" in address_detail:    # Jika mengandung kata "bali"
            return "Bali Vacation Rental"
        elif "nusa penida" in address_detail:   # Jika mengandung kata "nusa penida"
            return "Nusa Penida Vacation Rental"
        elif "yogyakarta" in address_detail:    # Jika mengandung kata "yogyakarta"
            return "Yogyakarta Vacation Rental"
    return row['tags'] 

In [39]:
# Mengganti nilai kosong ('') pada kolom 'tags' dengan NaN untuk mempermudah identifikasi nilai kosong
bukit_vista_property['tags'] = bukit_vista_property['tags'].replace('', np.nan)

# Terapkan fungsi ke DataFrame kolom 'tags'
bukit_vista_property['tags'] = bukit_vista_property.apply(fill_tags, axis=1)

In [ ]:
# mengklasifikasikan jenis properti berdasarkan kata-kata kunci tertentu yang terdapat dalam 'property_type'
def extract_class_property(property_type):
    property_type = property_type.lower()
    if 'villa' in property_type:
        return 'villa'
    elif 'guest house' in property_type:
        return 'guest house'
    elif 'view' in property_type:
        return 'view'
    return None

In [41]:
# Menambahkan kolom baru 'class' untuk mengelompokkan hasil scraping images
bukit_vista_property['class'] = bukit_vista_property['property_type'].apply(extract_class_property)

In [ ]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    # Menghapus spasi kosong di awal dan akhir teks
    text = text.strip()
    # Hapus URL, username, dan karakter non-alphanumeric -> regex
    text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text)
    # Hapus angka
    text = re.sub(r"\d+", "", text)
    # Hapus karakter spesial
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Memecah teks menjadi kata-kata (tokenisasi)
    tokens = word_tokenize(text)
    # Lematisasi untuk mendapatkan bentuk dasar kata
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Menghapus kata-kata yang dianggap tidak penting (stopwords)
    tokens = [word for word in tokens if word not in stop_words]
    # Mengubah kata menjadi bentuk dasarnya menggunakan stemming
    tokens = [ps.stem(word) for word in tokens]
    # Gabungkan kembali tokens menjadi teks
    text = ' '.join(tokens)
    # Mengembalikan text
    return text

In [ ]:
# membersihkan teks dalam kolom 'tags'
def clean_tags(text):
    # Ganti semua pola "\n\t" atau kombinasi "\n" dan "\t" dengan spasi
    text = re.sub(r"\s+", " ", text)  # Menghapus semua whitespace berlebih
    text = text.strip()  # Menghapus whitespace di awal dan akhir teks
    return text   # Mengembalikan text

In [44]:
# Fungsi untuk membersihkan teks dalam kolom 'property_id'
def clean_property_id(text):
    text = text.lower()  # Mengubah teks menjadi huruf kecil
    text = text.strip()  # Menghapus whitespace di awal dan akhir teks
    return text  # Mengembalikan text

In [ ]:
# Fungsi untuk membersihkan teks dalam kolom 'price'
def extract_price(text):
    match = re.search(r"\b\d+\b", text) 
    if match:
        return int(match.group())  
    return None 

In [ ]:
bukit_vista_df = bukit_vista_property.copy()

In [ ]:
bukit_vista_df

,property_links,title,address,tags,property_id,price,bedroom,room,bathroom,garage,property_type,property_status,guest_number,address_detail,city,state,zip_code,area,country,class
0,https://www.bukitvista.com/property/uluwatu-modern-boho-villa-near-nyang-nyang-beach-2,None,None,Bali Vacation Rental,NaN,Start from $65 USD per night,NaN,NaN,NaN,NaN,Villa,NaN,NaN,"Uluwatu, Bali",None,None,None,None,None,villa
1,https://www.bukitvista.com/property/bingin-beach-hideaway-group-villa-with-pool-amp-bbq,Bingin Beach Hideaway: Group Villa with Pool & BBQ,"Jl. Pantai Cemongkak Gg. Samuh Sari No.2, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali 80361",Featured\n Bali Vacation Rental\n \n Group Friendly Stay\n \n Long Stay Trend Property\n \n New Listing\n \n Private\n \n Private Pool\n \n Top Trending\n \n New,PR0874,Starting from USD 161 per night,3,3,3,1,"Amazing pool, Island life, Pool view, Surfing, Tropical, Villa","Bali Vacation Rental, Group Friendly Stay, Long Stay Trend Property, New Listing, Private, Private Pool, Top Trending",6,"Jl. Pantai Cemongkak Gg. Samuh Sari No.2, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Bingin,Indonesia,villa
2,https://www.bukitvista.com/property/mediterranean-luxury-villa-ubud-rooftop,4-Bedroom Mediterranean Luxury Villa with Ubud Rooftop View,"Jl. Suweta, Ubud, Kecamatan Ubud, Kabupaten Gianyar, Bali 80571",Featured\n Bali Vacation Rental\n \n Group Friendly Stay\n \n Instagramable Stay\n \n Long Stay Trend Property\n \n Private\n \n Private Pool\n \n Top Trending\n \n New,PR0960,USD 202 / Night,4,4,4,1,"Amazing pool, Amazing View, Jungle View, Pool view, Tropical, Villa","Bali Vacation Rental, Group Friendly Stay, Instagramable Stay, Long Stay Trend Property, Private, Private Pool, Top Trending",8,"Jl. Suweta, Ubud, Kecamatan Ubud, Kabupaten Gianyar, Bali",Bali,Bali,80571,Ubud,Indonesia,villa
3,https://www.bukitvista.com/property/nusa-dua-seafront-villa,Luxurious 3-Bedroom Nusa Dua Seafront Villa w/ Private Beach,None,Featured\n Bali Vacation Rental\n \n Top Trending,PR0242,Starting from USD 715 per 2 nights,3,NaN,4,NaN,"Beachfront, Villa","Bali Vacation Rental, Top Trending",6,"Jl. Raya Nusa Dua Selatan, Sawangan, Nusa Dua, Kabupaten Badung, Bali 80363",Bali,Bali,80363,Nusa Dua,Indonesia,villa
4,https://www.bukitvista.com/property/grand-villa-retreat-w-pool-amp-garden-in-ungasan,Grand Villa Retreat w/ Pool & Garden in Ungasan,"Jl. Pantai Balangan I No.9x, Ungasan, Kec. Kuta Sel., Kabupaten Badung, Bali 80361",Featured\n Group Friendly Stay\n \n Long Stay Trend Property\n \n New Listing\n \n Private Pool\n \n Top Trending\n \n New,PR0857,Starting from USD 84 per night,2,NaN,2,NaN,"Amazing pool, Golfing, Pool view, Villa","Group Friendly Stay, Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,"Jl. Pantai Balangan I No.9x, Ungasan, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Balangan,Indonesia,villa
5,https://www.bukitvista.com/property/surfers-villa-4-mins-to-bingin-dreamland-beach-by-bukit-vista,Surfer’s Villa 4 Mins to Bingin & Dreamland Beach by Bukit Vista,"Pecatu, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",Featured\n Long Stay Trend Property\n \n New Listing\n \n Private Pool\n \n Top Trending,PR0925,Starting from USD 118 per night,2,NaN,2.5,NaN,"Pool view, Villa","Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,"Jl. Bangbang Metuug, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali",Bali,Bali,80361,Pecatu,Indonesia,villa
6,https://www.bukitvista.com/property/private-pool-villa-minutes-from-bingin-surf-haven-by-bukit-vista,Private Pool Villa Minutes from Bingin Surf Haven by Bukit Vista,"Pecatu, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",Featured\n Long Stay Trend Property\n \n New Listing\n \n Private Pool\n \n Top Trending,PR0925,Starting from USD 165 per night,2,NaN,2,NaN,"Pool view, Villa","Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,"Jl. Bangbang Metuug, Pecatu, Kec. Kuta Sel

In [50]:
bukit_vista_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_links   51 non-null     object
 1   title            50 non-null     object
 2   address          38 non-null     object
 3   tags             51 non-null     object
 4   property_id      50 non-null     object
 5   price            51 non-null     object
 6   bedroom          50 non-null     object
 7   room             12 non-null     object
 8   bathroom         50 non-null     object
 9   garage           2 non-null      object
 10  property_type    51 non-null     object
 11  property_status  46 non-null     object
 12  guest_number     43 non-null     object
 13  address_detail   51 non-null     object
 14  city             49 non-null     object
 15  state            49 non-null     object
 16  zip_code         40 non-null     object
 17  area             48 non-null     obje

In [ ]:
# preprocess_text pada kolom title
bukit_vista_df = bukit_vista_df.dropna(subset=["title"])
bukit_vista_df["title"] = bukit_vista_df["title"].fillna("")
bukit_vista_df['title'] = bukit_vista_df['title'].apply(preprocess_text)

# preprocess_text pada kolom property_type
bukit_vista_df['property_type'] = bukit_vista_df['property_type'].apply(preprocess_text)

# preprocess_text pada kolom address_detail
bukit_vista_df['address_detail'] = bukit_vista_df['address_detail'].apply(preprocess_text)

# clean_tags pada kolom tags
bukit_vista_df['tags'] = bukit_vista_df['tags'].apply(clean_tags)

# preprocess_text pada kolom tags
bukit_vista_df['tags'] = bukit_vista_df['tags'].apply(preprocess_text)

# extract_price pada kolom price
bukit_vista_df['price'] = bukit_vista_df['price'].apply(extract_price)

# clean_property_id pada kolom property_id
bukit_vista_df['property_id'] = bukit_vista_df['property_id'].apply(clean_property_id)

<ipython-input-52-980e1247da52>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bukit_vista_df["title"] = bukit_vista_df["title"].fillna("")
<ipython-input-52-980e1247da52>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bukit_vista_df['title'] = bukit_vista_df['title'].apply(preprocess_text)
<ipython-input-52-980e1247da52>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [53]:
bukit_vista_df

,property_links,title,address,tags,property_id,price,bedroom,room,bathroom,garage,property_type,property_status,guest_number,address_detail,city,state,zip_code,area,country,class
1,https://www.bukitvista.com/property/bingin-beach-hideaway-group-villa-with-pool-amp-bbq,bingin beach hideaway group villa pool bbq,"Jl. Pantai Cemongkak Gg. Samuh Sari No.2, Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali 80361",featur bali vacat rental group friendli stay long stay trend properti new list privat privat pool top trend new,pr0874,161,3,3,3,1,amaz pool island life pool view surf tropic villa,"Bali Vacation Rental, Group Friendly Stay, Long Stay Trend Property, New Listing, Private, Private Pool, Top Trending",6,jl pantai cemongkak gg samuh sari pecatu kec kuta sel kabupaten badung bali,Bali,Bali,80361,Bingin,Indonesia,villa
2,https://www.bukitvista.com/property/mediterranean-luxury-villa-ubud-rooftop,bedroom mediterranean luxuri villa ubud rooftop view,"Jl. Suweta, Ubud, Kecamatan Ubud, Kabupaten Gianyar, Bali 80571",featur bali vacat rental group friendli stay instagram stay long stay trend properti privat privat pool top trend new,pr0960,202,4,4,4,1,amaz pool amaz view jungl view pool view tropic villa,"Bali Vacation Rental, Group Friendly Stay, Instagramable Stay, Long Stay Trend Property, Private, Private Pool, Top Trending",8,jl suweta ubud kecamatan ubud kabupaten gianyar bali,Bali,Bali,80571,Ubud,Indonesia,villa
3,https://www.bukitvista.com/property/nusa-dua-seafront-villa,luxuri bedroom nusa dua seafront villa w privat beach,None,featur bali vacat rental top trend,pr0242,715,3,NaN,4,NaN,beachfront villa,"Bali Vacation Rental, Top Trending",6,jl raya nusa dua selatan sawangan nusa dua kabupaten badung bali,Bali,Bali,80363,Nusa Dua,Indonesia,villa
4,https://www.bukitvista.com/property/grand-villa-retreat-w-pool-amp-garden-in-ungasan,grand villa retreat w pool garden ungasan,"Jl. Pantai Balangan I No.9x, Ungasan, Kec. Kuta Sel., Kabupaten Badung, Bali 80361",featur group friendli stay long stay trend properti new list privat pool top trend new,pr0857,84,2,NaN,2,NaN,amaz pool golf pool view villa,"Group Friendly Stay, Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,jl pantai balangan x ungasan kec kuta sel kabupaten badung bali,Bali,Bali,80361,Balangan,Indonesia,villa
5,https://www.bukitvista.com/property/surfers-villa-4-mins-to-bingin-dreamland-beach-by-bukit-vista,surfer villa min bingin dreamland beach bukit vista,"Pecatu, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",featur long stay trend properti new list privat pool top trend,pr0925,118,2,NaN,2.5,NaN,pool view villa,"Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,jl bangbang metuug pecatu kec kuta sel kabupaten badung bali,Bali,Bali,80361,Pecatu,Indonesia,villa
6,https://www.bukitvista.com/property/private-pool-villa-minutes-from-bingin-surf-haven-by-bukit-vista,privat pool villa minut bingin surf bukit vista,"Pecatu, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",featur long stay trend properti new list privat pool top trend,pr0925,165,2,NaN,2,NaN,pool view villa,"Long Stay Trend Property, New Listing, Private Pool, Top Trending",4,jl bangbang metuug pecatu kec kuta sel kabupaten badung bali,Bali,Bali,80361,Pecatu,Indonesia,villa
7,https://www.bukitvista.com/property/ungasan-exquisite-villa-w-rooftop-amp-private-pool-by-bukit-vista,ungasan exquisit villa w rooftop privat pool bukit vista,"Ungasan, Kuta Selatan, Badung, Bali, Nusa Tenggara, Indonesia",featur group friendli stay new list privat pool top trend,pr0936,167,3,NaN,3.5,NaN,amaz pool golf surf tropic villa,"Group Friendly Stay, New Listing, Private Pool, Top Trending",6,jl pesakapan indah ungasan kec kuta sel kabupaten badung bali,Bali,Bali,80361,Ungasan,Indonesia,villa
8,https://www.bukitvista.com/property/sun-soaked-canggu-villa-perfect-for-families,sun soak canggu villa perfect famili,"Tibubeneng, Kuta Utara, Badung, Bali, Nusa Tenggara, Indonesia",featur 

In [55]:
bukit_vista_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 1 to 50
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_links   50 non-null     object
 1   title            50 non-null     object
 2   address          38 non-null     object
 3   tags             50 non-null     object
 4   property_id      50 non-null     object
 5   price            50 non-null     int64 
 6   bedroom          50 non-null     object
 7   room             12 non-null     object
 8   bathroom         50 non-null     object
 9   garage           2 non-null      object
 10  property_type    50 non-null     object
 11  property_status  46 non-null     object
 12  guest_number     43 non-null     object
 13  address_detail   50 non-null     object
 14  city             49 non-null     object
 15  state            49 non-null     object
 16  zip_code         40 non-null     object
 17  area             48 non-null     object
 1

In [56]:
# Mengubah tipe data kolom 'price' menjadi string
bukit_vista_df['price'] = bukit_vista_df['price'].astype(str)

<ipython-input-56-6c758e6640fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bukit_vista_df['price'] = bukit_vista_df['price'].astype(str)


In [57]:
bukit_vista_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 1 to 50
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_links   50 non-null     object
 1   title            50 non-null     object
 2   address          38 non-null     object
 3   tags             50 non-null     object
 4   property_id      50 non-null     object
 5   price            50 non-null     object
 6   bedroom          50 non-null     object
 7   room             12 non-null     object
 8   bathroom         50 non-null     object
 9   garage           2 non-null      object
 10  property_type    50 non-null     object
 11  property_status  46 non-null     object
 12  guest_number     43 non-null     object
 13  address_detail   50 non-null     object
 14  city             49 non-null     object
 15  state            49 non-null     object
 16  zip_code         40 non-null     object
 17  area             48 non-null     object
 1

In [58]:
bukit_vista_df.to_csv("data_scraping.csv", index=False)